In [25]:
import numpy as np
import torch
import pandas as pd
import math
from torch import nn


In [19]:
articles = pd.read_csv("Data/articles.csv")
behaviors_train = pd.read_csv("Data/behaviors_train.csv")
behaviors_val = pd.read_csv("Data/behaviors_val.csv")
history_train = pd.read_csv("Data/history_train.csv")
history_val = pd.read_csv("Data/history_val.csv")

**Main idea behind treating the attributes cyclic** Values close on the cycle have similar sine and cosine representations.

**Example** After 23:59 comes 00:00, so 23 and 0 are closer in time than 23 and 12.

**Calculations** Cyclic encoding solves the discontinuity problem by mapping cyclic values to points on a unit circle using sine and cosine functions:

$x_\text{sin} = \sin\left( \frac{2\pi \cdot \text{value}}{\text{cycle length}} \right)$

$x_\text{cos} = \cos\left( \frac{2\pi \cdot \text{value}}{\text{cycle length}} \right)$

In [23]:
def encode_timestamp(timestamp):
    dt = pd.to_datetime(timestamp)
    
    # Cyclic encoding
    sin_hour = np.sin(2 * np.pi * dt.hour / 24)
    cos_hour = np.cos(2 * np.pi * dt.hour / 24)
    
    # Continuous and one-hot features
    month_one_hot = np.eye(12)[dt.month - 1]  # One-hot encode month
    weekday_one_hot = np.eye(7)[dt.weekday()] # One-hot encode weekday

    # Combine features
    return np.concatenate([month_one_hot, weekday_one_hot, [sin_hour, cos_hour]])


In [24]:
timestamp = "2023-05-30 14:21:34"
embedding = encode_timestamp(timestamp)
print(embedding)
print(len(embedding))

[ 0.         0.         0.         0.         1.         0.
  0.         0.         0.         0.         0.         0.
  0.         1.         0.         0.         0.         0.
  0.        -0.5       -0.8660254]
21


## Inspiration: Code for implementation

Event-time joint embedding -> join time and title representation

Other source: https://github.com/huggingface/diffusers/blob/v0.11.0/src/diffusers/models/embeddings.py

In [ ]:
class TimestampEmbedding(nn.Module):
    def __init__(self, timestamp_dim):
        super().__init__()
        self.timestamp_dim = timestamp_dim
        self.fc = nn.Linear(timestamp_dim, timestamp_dim)  # Optional transformation

    def forward(self, timestamp):
        # `timestamp` should already be encoded as a feature vector
        return self.fc(timestamp)

In [ ]:
class CombinedEmbedding(nn.Module):
    def __init__(self, article_dim, timestamp_dim):
        super().__init__()
        self.fc = nn.Linear(article_dim + timestamp_dim, article_dim)  # Project back to article_dim

    def forward(self, article_emb, timestamp_emb):
        combined_emb = torch.cat([article_emb, timestamp_emb], dim=-1)
        return self.fc(combined_emb)  # Optionally project back

In [ ]:
class SelfAttention(nn.Module):
    def __init__(self, embed_dim, num_heads):
        super().__init__()
        self.attention = nn.MultiheadAttention(embed_dim=embed_dim, num_heads=num_heads)

    def forward(self, x):
        # x: [seq_len, batch_size, embed_dim]
        return self.attention(x, x, x)

In [ ]:
class NRMSWithTimestamps(nn.Module):
    def __init__(self, article_dim, timestamp_dim, embed_dim, num_heads):
        super().__init__()
        self.timestamp_embedding = TimestampEmbedding(timestamp_dim)
        self.combined_embedding = CombinedEmbedding(article_dim, timestamp_dim)
        self.self_attention = SelfAttention(embed_dim, num_heads)

    def forward(self, article_emb, timestamp):
        # Generate timestamp embeddings
        timestamp_emb = self.timestamp_embedding(timestamp)

        # Combine article and timestamp embeddings
        combined_emb = self.combined_embedding(article_emb, timestamp_emb)

        # Apply self-attention
        attention_out, _ = self.self_attention(combined_emb)

        return attention_out